In [1]:
data('PlantGrowth')
head(PlantGrowth)

,weight,group
,<dbl>,<fct>
1,4.17,ctrl
2,5.58,ctrl
3,5.18,ctrl
4,6.11,ctrl
5,4.50,ctrl
6,4.61,ctrl


In [2]:
levels(PlantGrowth$group)     #그룹에 몇 개의 수준이 있는지 확인
#'ctrl', 'trt1', 'trt2'
anova1 = aov(weight ~ group, data = PlantGrowth)
summary(anova1)

[1] "ctrl" "trt1" "trt2"

            Df Sum Sq Mean Sq F value Pr(>F)  
group        2  3.766  1.8832   4.846 0.0159 *
Residuals   27 10.492  0.3886                 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

In [9]:
shapiro.test(PlantGrowth$weight[PlantGrowth$group=='ctrl'])
shapiro.test(PlantGrowth$weight[PlantGrowth$group=='trt1'])
shapiro.test(PlantGrowth$weight[PlantGrowth$group=='trt2'])


	Shapiro-Wilk normality test

data:  PlantGrowth$weight[PlantGrowth$group == "ctrl"]
W = 0.95668, p-value = 0.7475



	Shapiro-Wilk normality test

data:  PlantGrowth$weight[PlantGrowth$group == "trt1"]
W = 0.93041, p-value = 0.4519



	Shapiro-Wilk normality test

data:  PlantGrowth$weight[PlantGrowth$group == "trt2"]
W = 0.94101, p-value = 0.5643


In [3]:
library(lawstat)
bartlett.test(PlantGrowth$weight, PlantGrowth$group)
#바틀렛 검정
levene.test(PlantGrowth$weight, PlantGrowth$group, location='mean')
#레빈 검정


	Bartlett test of homogeneity of variances

data:  PlantGrowth$weight and PlantGrowth$group
Bartlett's K-squared = 2.8786, df = 2, p-value = 0.2371



	Classical Levene's test based on the absolute deviations from the mean
	( none not applied because the location is not set to median )

data:  PlantGrowth$weight
Test Statistic = 1.237, p-value = 0.3062


In [7]:
install.packages('DescTools')
library(DescTools)
PostHocTest(anova1, method="lsd")
PostHocTest(anova1, method="bonferroni")
PostHocTest(anova1, method="hsd")
PostHocTest(anova1, method="scheffe")

Installing package into 'C:/Users/82107/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

also installing the dependencies 'rootSolve', 'lmom', 'expm', 'Exact', 'gld'




package 'rootSolve' successfully unpacked and MD5 sums checked
package 'lmom' successfully unpacked and MD5 sums checked
package 'expm' successfully unpacked and MD5 sums checked
package 'Exact' successfully unpacked and MD5 sums checked
package 'gld' successfully unpacked and MD5 sums checked
package 'DescTools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\82107\AppData\Local\Temp\Rtmpqk203t\downloaded_packages



  Posthoc multiple comparisons of means : Fisher LSD 
    95% family-wise confidence level

$group
            diff      lwr.ci    upr.ci   pval    
trt1-ctrl -0.371 -0.94301261 0.2010126 0.1944    
trt2-ctrl  0.494 -0.07801261 1.0660126 0.0877 .  
trt2-trt1  0.865  0.29298739 1.4370126 0.0045 ** 

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



  Posthoc multiple comparisons of means : Bonferroni 
    95% family-wise confidence level

$group
            diff     lwr.ci    upr.ci   pval    
trt1-ctrl -0.371 -1.0825786 0.3405786 0.5832    
trt2-ctrl  0.494 -0.2175786 1.2055786 0.2630    
trt2-trt1  0.865  0.1534214 1.5765786 0.0134 *  

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



  Posthoc multiple comparisons of means : Tukey HSD 
    95% family-wise confidence level

$group
            diff     lwr.ci    upr.ci   pval    
trt1-ctrl -0.371 -1.0622161 0.3202161 0.3909    
trt2-ctrl  0.494 -0.1972161 1.1852161 0.1980    
trt2-trt1  0.865  0.1737839 1.5562161 0.0120 *  

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



  Posthoc multiple comparisons of means: Scheffe Test 
    95% family-wise confidence level

$group
            diff     lwr.ci    upr.ci   pval    
trt1-ctrl -0.371 -1.0930531 0.3510531 0.4241    
trt2-ctrl  0.494 -0.2280531 1.2160531 0.2265    
trt2-trt1  0.865  0.1429469 1.5870531 0.0163 *  

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
